### feature합치기
#### investors, historical(매일 거래량 등) and 분류항목 합치기

In [1]:
import datetime, time
from datetime import date

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler

In [2]:
code = {'005930': ['삼성전자', 'sec'], '005380': ['현대차', 'hyunmotor'],
                 '035420': ['NAVER', 'naver'], '033780': ['KT&G', 'ktng']}
# code = {'005930': ['삼성전자', 'sec']}

In [3]:
pkl_directory = '../data/company_daily_price_pkl'
pkl_directory = './'
pkl_directory = '../data/company_pkl/'

In [4]:
# investor.pkl, historical.pkl. 읽기
for key, val in code.items():
    pkl_name= '{}.pkl'.format(val[1])
    globals()['df_{}'.format(val[1])] = pd.read_pickle('./' + pkl_name)
    globals()['df_{}'.format(val[1])]['date'] = pd.to_datetime(globals()['df_{}'.format(val[1])]['date']) #change to timestamp
    globals()['df_{}'.format(val[1])]['date'] = globals()['df_{}'.format(val[1])]['date'].dt.date  # change to datetime
    pkl_name= '{}_historical.pkl'.format(val[1])
    globals()['df_{}_{}'.format(val[1], 'historical')] = pd.read_pickle(pkl_directory + pkl_name)
    globals()['df_{}_{}'.format(val[1], 'historical')]['date'] = globals()['df_{}_{}'.format(val[1], 'historical')]['date'].dt.date # change to datetime
    pkl_name= '{}_investors.pkl'.format(val[1])
    globals()['df_{}_{}'.format(val[1], 'investors')] = pd.read_pickle(pkl_directory + pkl_name)
    globals()['df_{}_{}'.format(val[1], 'investors')]['date'] = globals()['df_{}_{}'.format(val[1], 'investors')]['date'].dt.date # change to datetime

In [5]:
# index column : date
def find_previous_date(df_inv, df_his, date_current):
    i = 0
    date2 = 0
    while True:
        i += 1
        date2 = date_current - datetime.timedelta(days = i)
        if (df_inv.loc[date2]['retail'] != 0) & (df_his.loc[date2]['vol'] != '') :  
            # investors와 historical 두 개 데이터에 같이 없어야 함
            break
        if i >= 30:
            break
    return date2

In [6]:
def find_ratio(df_o, date_current):
    df_o_trans = df_o.transpose()
    df_o_trans.columns = ['before', 'after']
    df_o_trans[date_current] = (df_o_trans['after'] - df_o_trans['before']) / df_o_trans['before']
    return df_o_trans.transpose()

In [71]:
df_inv = df_sec_investors.set_index('date')
df_his = df_sec_historical.set_index('date')
df_o = df_sec.set_index('date')

In [27]:
type(df_o.index[0]), type(df_inv.index[0]), type(df_his.index[0])

(datetime.date, datetime.date, datetime.date)

In [58]:
date_current = df_o.index[0]
date_previous = find_previous_date(df_inv, df_his, date_current)
date_previous_1 = find_previous_date(df_inv, df_his, date_previous)
df_inv_comp = df_inv.loc[[date_previous_1, date_previous]]
df_his_comp = df_his.loc[[date_previous_1, date_previous]]

In [60]:
aa = find_ratio(df_inv_comp, date_current)
aa.iloc[-1]

,retail,foreigner,institution,financial,invtrust,pension,privequity,bank,insurance,financeetc,corporateetc,foreigneretc
before,42096.000000,-33925.000000,-13756.000000,12591.000000,913.000000,-21200.000000,-451.00000,-1802.000000,-867.000000,-2941.00000,4387.000000,1199.000000
after,387097.000000,-438334.000000,36417.000000,105301.000000,-27017.000000,-28510.000000,-11227.00000,3268.000000,-6844.000000,1446.00000,12524.000000,2296.000000
2022-06-20,8.195577,11.920678,-3.647354,7.363196,-30.591457,0.344811,23.89357,-2.813541,6.893887,-1.49167,1.854798,0.914929


In [79]:
aa.iloc[-1]

retail           8.195577
foreigner       11.920678
institution     -3.647354
financial        7.363196
invtrust       -30.591457
pension          0.344811
privequity      23.893570
bank            -2.813541
insurance        6.893887
financeetc      -1.491670
corporateetc     1.854798
foreigneretc     0.914929
Name: 2022-06-20, dtype: float64

In [80]:
aa1.iloc[-1]

retail         -0.674957
foreigner      -0.407361
institution     2.593047
financial       0.295068
invtrust       -0.938927
pension        -0.883900
privequity     -0.821502
bank           -0.937576
insurance      -0.955874
financeetc      0.062932
corporateetc   -0.814356
foreigneretc   -0.661150
Name: 2022-06-21, dtype: float64

In [61]:
aa1

,retail,foreigner,institution,financial,invtrust,pension,privequity,bank,insurance,financeetc,corporateetc,foreigneretc
before,387097.000000,-438334.000000,36417.000000,105301.000000,-27017.000000,-28510.0000,-11227.000000,3268.000000,-6844.000000,1446.000000,12524.000000,2296.00000
after,125823.000000,-259774.000000,130848.000000,136372.000000,-1650.000000,-3310.0000,-2004.000000,204.000000,-302.000000,1537.000000,2325.000000,778.00000
2022-06-21,-0.674957,-0.407361,2.593047,0.295068,-0.938927,-0.8839,-0.821502,-0.937576,-0.955874,0.062932,-0.814356,-0.66115


In [72]:
df_o = pd.merge(df_o, aa, left_index=True, right_index=True, how='left')
df_o

,value,retail,foreigner,institution,financial,invtrust,pension,privequity,bank,insurance,financeetc,corporateetc,foreigneretc
date,,,,,,,,,,,,,
2022-06-20,2,8.195577,11.920678,-3.647354,7.363196,-30.591457,0.344811,23.89357,-2.813541,6.893887,-1.49167,1.854798,0.914929
2022-06-21,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-22,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-23,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-24,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-06,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-02-07,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-02-08,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
bb1 = aa1.iloc[-1]

In [76]:
aa1.index[-1]

datetime.date(2022, 6, 21)

In [77]:
df_o.loc[aa1.index[-1]] = bb1

In [78]:
df_o

,value,retail,foreigner,institution,financial,invtrust,pension,privequity,bank,insurance,financeetc,corporateetc,foreigneretc
date,,,,,,,,,,,,,
2022-06-20,2.0,8.195577,11.920678,-3.647354,7.363196,-30.591457,0.344811,23.893570,-2.813541,6.893887,-1.491670,1.854798,0.914929
2022-06-21,NaN,-0.674957,-0.407361,2.593047,0.295068,-0.938927,-0.883900,-0.821502,-0.937576,-0.955874,0.062932,-0.814356,-0.661150
2022-06-22,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-23,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-24,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-06,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-02-07,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-02-08,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
bb = pd.concat([df_o, aa1], axis=0)
bb

,value,retail,foreigner,institution,financial,invtrust,pension,privequity,bank,insurance,financeetc,corporateetc,foreigneretc
2022-06-20,2.0,8.195577,11.920678,-3.647354,7.363196,-30.591457,0.344811,23.893570,-2.813541,6.893887,-1.491670,1.854798,0.914929
2022-06-21,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-22,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-23,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-24,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-09,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-02-10,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
before,NaN,387097.000000,-438334.000000,36417.000000,105301.000000,-27017.000000,-28510.000000,-11227.000000,3268.000000,-6844.000000,1446.000000,12524.000000,2296.000000
after,NaN,125823.000000,-259774.000000,130848.000000,136372.000000,-1650.000000,-3310.000000,-2004.000000,204.000000,-302.000000,1537.000000,2325.000000,778.000000


In [69]:
df_o.join(aa1, how='left')

ValueError: columns overlap but no suffix specified: Index(['retail', 'foreigner', 'institution', 'financial', 'invtrust',
       'pension', 'privequity', 'bank', 'insurance', 'financeetc',
       'corporateetc', 'foreigneretc'],
      dtype='object')

In [21]:
type(df_o.index[0])

datetime.date

In [ ]:
df_o.join(aa)

In [ ]:
bb = date_current.strftime('%Y-%m-%d')
bb

In [ ]:
result4 = pd.con([df1,sr1], axis=1)

In [ ]:
df_sec_investors['date'][0]

In [ ]:
df_inv_trans = df_inv_comp.transpose()
df_inv_trans.columns = ['before', 'after']
df_inv_trans['perc'] = (df_inv_trans['after'] - df_inv_trans['before']) / df_inv_trans['before']
df_inv = df_inv_trans.transpose()

In [ ]:
df_inv_trans['perc'] = (df_inv_trans['after'] - df_inv_trans['before']) / df_inv_trans['before']

In [ ]:
df_inv = df_inv_trans.transpose()

In [ ]:
def calc_percent(x, y) :
    return (y - x) / x

In [ ]:
del df_nn
df_nn = df_inv_trans

In [ ]:
df_nn ['perc'] = (df_nn['after'] - df_nn['before']) / df_nn['before']

In [ ]:
df_nn

In [ ]:
df_inv_trans.columns = ['before', 'after']

In [ ]:
df_inv_comp.apply()

In [ ]:
def calc_increment_percent(df_inv, df_his, date_current):
    date_previous = find_previous_date(df_inv, df_his, date_current)
    df_inv_comp = df_inv.loc[[date_previous, date_current]]
    df_his_comp = df_his.loc[[date_previous, date_current]]
    return inc

In [ ]:
date1 = datetime.datetime.strptime(df_sec.loc[0, 'date'], '%Y-%m-%d')
# date1 = datetime.datetime.strptime('2022-06-16', '%Y-%m-%d')
aa = find_previous_date(df_inv, df_his, date1)

In [ ]:
aa

In [ ]:
date1 = datetime.datetime.strptime('2022-06-17', '%Y-%m-%d')
bb = find_previous_date(df_inv, df_his, date1)
bb